# Part 1: LeNet on MINST

In [1]:
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D

from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

We can check the shape of new data and see that our images are 28×28 pixels, so we need to add a new axis, which will represent a number of channels. Also, it is important to do one-hot encoding of labels and normalization of input images.

In [4]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0].shape, 'image shape')

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples
(28, 28) image shape


In [5]:
# Add a new axis
x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0].shape, 'image shape')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(28, 28, 1) image shape


In [6]:
# Convert class vectors to binary class matrices.

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [7]:
# Data normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Now, it is time to start using TensorFlow 2.0 in order to build our convolutional neural network. The easiest way to do this is by using the Sequential API. We will wrap it in a class called LeNet. The input is an image, and the output will be a class probability vector.

In [8]:
# LeNet-5 model
class LeNet(Sequential):
    def __init__(self, input_shape, nb_classes):
        super().__init__()

        self.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape, padding="same"))
        self.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
        self.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Flatten())
        self.add(Dense(120, activation='tanh'))
        self.add(Dense(84, activation='tanh'))
        self.add(Dense(nb_classes, activation='softmax'))

        self.compile(optimizer='adam',
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])

In [9]:
model = LeNet(x_train[0].shape, num_classes)
model.summary()

Model: "le_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                10164

After creating a model, we need to train its parameters to make it powerful. Let’s train the model for a given number of epochs.


In [10]:
# Place the logs in a timestamped subdirectory
# This allows to easy select different training runs
# In order not to overwrite some data, it is useful to have a name with a timestamp
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Specify the callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

In [11]:
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/20
  1/938 [..............................] - ETA: 0s - loss: 2.3381 - accuracy: 0.1250WARNING:tensorflow:From /Users/zhangmaoyu/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 10s 11ms/step - loss: 0.5559 - accuracy: 0.7965 - val_loss: 0.4528 - val_accuracy: 0.8312
Epoch 2/20
938/938 [==============================] - 10s 11ms/step - loss: 0.3846 - accuracy: 0.8594 - val_loss: 0.3856 - val_accuracy: 0.8604
Epoch 3/20
938/938 [==============================] - 10s 11ms/step - loss: 0.3413 - accuracy: 0.8741 - val_loss: 0.3627 - val_accuracy: 0.8683
Epoch 4/20
938/938 [==============================] - 11s 11ms/step - loss: 0.3142 - accuracy: 0.8847 - val_loss: 0.3430 - val_accuracy: 0.8770
Epoch 5/20
938/938 [=====================

In [12]:
%tensorboard --logdir=logs/fit

Next,we will build Lenet by karas to compare our own lenet by training accuracy and loss,and testing accuracy and loss.

## Using karas to define model

In [19]:
import os
import datetime
import tensorflow as tf

In [20]:
# Load the digit dataset
mnist = tf.keras.datasets.mnist

# Spliting data into train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [21]:
# Creating a model
def create_model():
    
    model = tf.keras.models.Sequential()
    
    # Flatten - Flattens the input. Does not affect the batch size.
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    # Dense - Fully conenected layer ( Each Input Neuron is connected to the output Neuron)
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    # Dropout is a technique used to improve over-fit on neural networks
    # Basically during training some of neurons on a particular layer will be deactivated.
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model

In [22]:
# Hyperparameters
training_epochs = 20 # Total number of training epochs
learning_rate = 0.001 # The learning rate


In [23]:
model = create_model()

# Configure model for training
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [24]:
# In order not to overwrite some data, place the logs in a timestamped subdirectory
# This allows to easy select different training runs
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# Specify the callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored 
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

model.fit(x=x_train, 
          y=y_train, 
          epochs=20, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2209 - accuracy: 0.9356 - val_loss: 0.1136 - val_accuracy: 0.9647
Epoch 2/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0962 - accuracy: 0.9711 - val_loss: 0.0745 - val_accuracy: 0.9763
Epoch 3/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0694 - accuracy: 0.9787 - val_loss: 0.0775 - val_accuracy: 0.9759
Epoch 4/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0523 - accuracy: 0.9836 - val_loss: 0.0692 - val_accuracy: 0.9801
Epoch 5/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0436 - accuracy: 0.9860 - val_loss: 0.0643 - val_accuracy: 0.9800
Epoch 6/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0363 - accuracy: 0.9880 - val_loss: 0.0738 - val_accuracy: 0.9794
Epoch 7/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0304 - accuracy: 0.9897 - val_loss: 0.0630 - val_accuracy:

In [25]:
%tensorboard --logdir=logs/fit

Reusing TensorBoard on port 6007 (pid 15207), started 0:14:03 ago. (Use '!kill 15207' to kill it.)

## conclusion：

So the last command will open a TensorBoard in our notebook，So here, we have visualized all the graphs together, including train and validation accuracy and loss over epochs on x-axis, and actual value on y-axis.

One of the advantages of creating TensorBoard records using the Sequential API is that we can easily visualize our model. 
For this, click on Graphs tab. By default, we will see an op-level graph, which helps us to understand how TensorFlow understands our program. Examining the op-level graph can give us insight as how to change our model.
On the Distributions dashboard. Here we can view the distribution of each weight. The lighter part shows all the weight across time and the shaded part shows weights that are actually activated.


By using networks built by Karas, we can see that our own networks are performing well,and not bad for only 20 epochs.The training accuracy is up to 98%,and the testing accuracy is about to 90%,so that using LeNet to train MINST is very well.Because this network is primarily designed for MNIST dataset, it performs significantly better on it .

http://datahacker.rs/lenet-5-implementation-tensorflow-2-0/